In [ ]:
import spacy
# Загрузка научной модели SciSpacy
# nlp_sci = spacy.load("en_core_sci_sm")
nlp = spacy.load("en_core_web_sm")

from pymongo import MongoClient
import tqdm
from transformers import pipeline


In [ ]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "true"

# Создание размеченного датасета для дообучения модели

In [68]:
categories = {
    "TECHNOLOGY": [
        'artificial intelligence', 'big data', 'internet of things (IoT)', 
        'cloud computing', 'quantum computing', 'edge computing', 
        'blockchain', '5g networks', 'augmented reality', 'virtual reality', 
        'serverless computing', 'digital twins', 'microservices', 
        'event-driven architecture', 'in-memory computing', 'fog computing', 
        'cyber-physical systems', 'software-defined networks', 
        'data lakehouse', 'data mesh', 'robotics process automation (RPA)',
        'human-computer interaction (HCI)', 'cybersecurity', 'distributed computing', 
        'neuro-symbolic AI', 'federated learning', 'privacy-preserving AI',
        'explainable AI (XAI)', 'multi-cloud environments', 'data observability',
        'green computing', 'energy-efficient AI', 'smart contracts',
        'autonomous systems', 'immersive technologies', 'zero-trust architecture',
        'context-aware computing', 'haptic technologies', 'nanotechnologies'
    ],
    "METHOD": [
        'principal component analysis (PCA)', 'bayesian methods', 
        'markov chains', 'gradient boosting', 'stochastic processes', 
        'time series decomposition', 'dynamic programming', 
        'feature engineering', 'dimensionality reduction', 
        'data sampling', 'data imputation', 'data augmentation', 
        'semi-supervised learning', 'self-supervised learning', 
        'meta-learning', 'few-shot learning', 'multi-task learning', 
        'transfer learning', 'ensemble learning', 
        'autoregressive integrated moving average (ARIMA)', 
        'exponential smoothing', 'kalman filtering', 'survival analysis', 
        'probabilistic graphical models', 'causal inference', 
        'topic modeling', 'outlier detection', 'active learning', 
        'neural architecture search', 'attention mechanisms', 
        'hierarchical clustering', 'contrastive learning',
        'adaptive boosting', 'bagging', 'maximum likelihood estimation',
        'expectation-maximization', 'manifold learning', 'spectral embedding',
        'regularization techniques', 'robust optimization', 
        'decision analysis', 'swarm intelligence'
    ],
    "ALGORITHM": [
        'k-means', 'decision trees', 'random forest', 'svm', 
        'naive bayes', 'knn', 'logistic regression', 'linear regression', 
        'genetic algorithms', 'gradient descent', 'simulated annealing', 
        'particle swarm optimization', 'hill climbing', 'pagerank', 
        'dijkstra algorithm', 'kruskal algorithm', 'prim algorithm', 
        'bipartite matching', 'shortest path algorithms', 
        'hierarchical clustering', 'dbscan', 'mean-shift clustering', 
        'spectral clustering', 'xgboost', 'lightgbm', 'catboost', 
        'deep q-learning', 'ppo (proximal policy optimization)', 
        'transformer models', 'variational inference', 
        'reinforcement learning algorithms', 'apriori', 'fp-growth', 
        'monte carlo tree search', 'long short-term memory networks (LSTM)',
        'convolutional neural networks (CNN)', 'self-organizing maps', 
        'word2vec', 'doc2vec', 'collaborative filtering algorithms', 
        'temporal difference learning', 'value iteration', 'policy gradient methods',
        'trust region policy optimization (TRPO)', 'asynchronous advantage actor-critic (A3C)'
    ],
    "TASK": [
        'classification', 'clustering', 'regression', 
        'dimensionality reduction', 'time series forecasting', 
        'anomaly detection', 'sentiment analysis', 'recommendation systems', 
        'topic modeling', 'translation', 'object detection', 
        'speech recognition', 'image segmentation', 'text summarization', 
        'document classification', 'entity recognition', 
        'causal modeling', 'explainable AI', 'automated feature extraction',
        'intelligent tutoring', 'robot path planning', 'game AI design', 
        'event detection', 'sequence labeling', 'action recognition',
        'domain adaptation', 'knowledge graph construction'
    ],
    "MODEL": [
        'linear regression', 'logistic regression', 'decision trees', 
        'random forest', 'k-nearest neighbors (kNN)', 'support vector machine (SVM)', 
        'naive bayes', 'multilayer perceptron (MLP)', 'convolutional neural networks (CNN)', 
        'recurrent neural networks (RNN)', 'transformers', 'bert', 
        'gpt', 'longformer', 'albert', 'graph neural networks (GNN)', 
        'autoencoders', 'generative adversarial networks (GAN)', 
        'tabular neural networks', 'variational autoencoders (VAE)', 
        'language models', 'vision transformers (ViT)', 'ensemble models', 
        'mixture of experts', 'meta-modeling', 'neuro-symbolic models',
        'zero-shot models', 'contrastive learning models', 'sequence-to-sequence models',
        'multi-modal transformers', 'attention-based models', 'biLSTMs', 'capsule networks'
    ],
    "TOOL": [
        'apache hadoop', 'apache spark', 'tensorflow', 'pytorch', 
        'scikit-learn', 'pandas', 'numpy', 'matplotlib', 
        'seaborn', 'plotly', 'd3.js', 'tableau', 'power bi', 
        'kafka', 'apache beam', 'apache flink', 'kubeflow', 
        'mlflow', 'grafana', 'superset', 'aws glue', 'databricks', 
        'nltk', 'spacy', 'gensim', 'flask', 'streamlit', 
        'fastapi', 'transformers library', 'huggingface datasets', 
        'langchain', 'ray tune', 'azure ml', 'google colab', 
        'anaconda', 'mlpack', 'jupyter lab', 'knime',
        'snowflake', 'airflow', 'datadog', 'matplotlib', 'dash', 
        'highcharts', 'supervised library', 'mlxtend'
    ],
    "FRAMEWORK": [
        'tensorflow', 'pytorch', 'keras', 'apache flink', 
        'apache storm', 'ray', 'dask', 'spark mllib', 
        'scikit-learn pipelines', 'fastai', 'hugging face transformers', 
        'langchain', 'ray tune', 'azure ml', 'google ai platform', 
        'kubeflow pipelines', 'mlflow tracking', 'microsoft synapse', 
        'amazon sagemaker', 'openvino', 'mlxtend', 'bigdl', 
        'deeplearning4j', 'onnx runtime', 'paddlepaddle'
    ],
    "DATA": [
        'structured data', 'unstructured data', 'semi-structured data', 
        'time series data', 'historical data', 'geospatial data', 
        'streaming data', 'synthetic data', 'metadata', 
        'bigquery datasets', 'delta lake datasets', 'data cubes', 
        'training data', 'test data', 'validation data', 
        'annotated datasets', 'graph data', 'relational data', 
        'vector embeddings', 'text corpora', 'speech data', 
        'event streams', 'tabular data', 'spatial-temporal datasets',
        'encrypted datasets', 'multi-view data'
    ],
    "PARAMETER": [
        'learning rate', 'batch size', 'number of layers', 
        'dropout rate', 'number of epochs', 'activation functions', 
        'optimizer settings', 'momentum', 'weight decay', 
        'regularization parameters', 'embedding size', 
        'sequence length', 'window size', 'attention heads', 
        'hidden layer size', 'decay rate', 'max iterations', 
        'gradient clipping', 'loss function', 'hyperparameter ranges', 
        'convergence thresholds'
    ],
    "METRIC": [
        'accuracy', 'precision', 'recall', 'f1-score', 
        'mean squared error (MSE)', 'root mean squared error (RMSE)', 
        'mean absolute error (MAE)', 'roc-auc score', 'log loss', 
        'silhouette score', 'calinski-harabasz index', 'perplexity', 
        'bleu score', 'rouge score', 'edit distance', 
        'cosine similarity', 'mean average precision (MAP)', 
        'dcg', 'ndcg', 'kendall rank correlation', 'pairwise precision', 
        'normalized mutual information (NMI)', 'jaccard index'
    ],
    "APPLICATION": [
        'predictive analytics', 'healthcare analytics', 'genomics', 
        'fraud detection', 'marketing analytics', 'recommender systems', 
        'financial analytics', 'cybersecurity', 'traffic prediction', 
        'robotics', 'autonomous vehicles', 'smart cities', 
        'natural language processing', 'time series forecasting', 
        'climate modeling', 'bioinformatics', 'customer lifetime value', 
        'gaming analytics', 'personalization systems', 'geospatial analysis', 
        'speech synthesis', 'virtual assistants', 'real-time analytics', 
        'demand forecasting', 'inventory optimization', 
        'e-commerce analytics', 'social media analytics', 
        'precision agriculture', 'renewable energy optimization', 
        'disaster management', 'financial fraud detection', 'edge AI applications'
    ]
}


In [ ]:
#  шаблоны предложений
templates = [
    "The study demonstrates how {} and {} can improve the efficiency of {} tasks.",
    "Using {}, the authors were able to achieve significant improvements in {} with metrics like {}.",
    "{} and {} are critical for {} and have been explored in depth in this research.",
    "This paper compares {} and {} for solving problems related to {}.",
    "Advanced techniques such as {} and {} have been utilized in applications like {}.",
    "The experimental setup involved using {} for tasks like {}, achieving notable {}.",
    "Recent advancements in {} have opened new possibilities for improving {}.",
    "{} has been combined with {} to create innovative solutions for {}.",
    "In the context of {}, methods such as {} and {} were extensively evaluated.",
    "Key contributions include the integration of {} with {} for enhanced {}.",
    "The authors present a novel approach combining {} and {} to tackle challenges in {}.",
    "Performance improvements were observed when {} was applied alongside {} for {}.",
    "The methodology employs {} and {} for robust {} pipelines.",
    "This approach highlights the synergy between {} and {} in enhancing {} outcomes.",
    "Case studies demonstrate the practical applications of {} and {} in {} scenarios."
]

In [ ]:
import random
import json
from tqdm import tqdm
import re

# Функции для нормализации и поиска терминов
def normalize_term(term):
    """Нормализует термин для учета регистра и различных вариаций."""
    return re.sub(r'\s+', ' ', term.strip().lower())

def term_variations(term):
    """
    Генерирует различные вариации термина.
    Например: "Principal Component Analysis" -> ["PCA", "principal component analysis"]
    """
    variations = [normalize_term(term)]
    # Добавляем аббревиатуры, если они указаны в скобках
    if '(' in term and ')' in term:
        match = re.search(r'\((.*?)\)', term)
        if match:
            abbr = match.group(1).lower()
            full_form = re.sub(r'\(.*?\)', '', term).strip().lower()
            variations.extend([abbr, full_form])
    return variations

# Обработка категорий с учетом вариаций
def process_categories(categories):
    processed = {}
    for category, terms in categories.items():
        expanded_terms = []
        for term in terms:
            expanded_terms.extend(term_variations(term))
        processed[category] = expanded_terms
    return processed

# Улучшенная функция для получения категории термина
def get_category(tokenized_term, categories):
    """
    Определяет категорию для токенизированного термина.
    """
    for category, terms in categories.items():
        for term in terms:
            term_tokens = term.split()
            if tokenized_term == term_tokens:
                return category
    return "O"

# Улучшенная аннотация текста
def annotate_text(text, terms, categories):
    tokens = text.split()
    labels = ["O"] * len(tokens)
    term_dict = {tuple(term.split()): category for category, term_list in categories.items() for term in term_list}

    # Ищем термины в тексте
    for i in range(len(tokens)):
        for length in range(1, min(4, len(tokens) - i + 1)):  # Проверяем термины длиной до 3 токенов
            span = tuple(normalize_term(' '.join(tokens[i:i + length])).split())
            if span in term_dict:
                category = term_dict[span]
                labels[i] = f"B-{category}"
                for j in range(1, length):
                    labels[i + j] = f"I-{category}"
    return {"tokens": tokens, "ner_tags": labels}

# Функция генерации случайного предложения
def generate_sentence(terms):
    # Убедимся, что количество выбираемых терминов не превышает длину списка
    max_terms = min(len(terms), 5)  # Максимум 5 терминов
    selected_terms = random.sample(terms, random.randint(3, max_terms))
    template = random.choice(templates)
    return template.format(*selected_terms)

# Генерация синтетического датасета
def generate_synthetic_dataset(categories, num_sentences=10000):
    processed_categories = process_categories(categories)
    terms = [term for terms_list in processed_categories.values() for term in terms_list]
    sentences = []
    print("Generating synthetic dataset with term variations...")
    for _ in tqdm(range(num_sentences)):
        # Проверяем, есть ли хотя бы 3 термина в списке
        if len(terms) < 3:
            raise ValueError("Insufficient terms in the list to generate sentences.")
        sentence = generate_sentence(terms)
        annotated = annotate_text(sentence, terms, processed_categories)
        sentences.append(annotated)
    return sentences


# Генерация датасета
synthetic_dataset = generate_synthetic_dataset(categories, num_sentences=10000)

# Сохранение в файл
with open("synthetic_ner_dataset.json", "w", encoding="utf-8") as f:
    json.dump(synthetic_dataset, f, indent=4)


# Процесс дообучения модели

In [1]:
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
import evaluate
from tqdm import tqdm

# Шаг 1. Загрузка синтетического размеченного датасета
with open("synthetic_ner_dataset.json", "r", encoding="utf-8") as f:
    synthetic_data = json.load(f)

# Преобразование в формат Hugging Face
def convert_to_hf_format(data):
    tokens = [entry["tokens"] for entry in data]
    ner_tags = [entry["ner_tags"] for entry in data]
    return {"tokens": tokens, "ner_tags": ner_tags}

hf_data = convert_to_hf_format(synthetic_data)

# Создание Dataset
dataset = Dataset.from_dict(hf_data)

# Разделение на обучающую и тестовую выборки
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")

# Шаг 2. Токенизация
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Уникальные тэги
unique_labels = list(set(tag for tags in hf_data["ner_tags"] for tag in tags))
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Токенизация и выравнивание меток
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",  # Добавляем автоматическое выравнивание
        max_length=512
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Связь токенов и слов
        aligned_labels = []
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)  # Игнорируем специальные токены ([CLS], [SEP])
            else:
                aligned_labels.append(label2id[label[word_id]])
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

# Шаг 3. Загрузка модели
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)

# Шаг 4. Метрики оценки
metric = evaluate.load("seqeval")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = predictions.argmax(-1)
    true_labels = [
        [id2label[label] for label in label_seq if label != -100]
        for label_seq in labels
    ]
    true_predictions = [
        [id2label[p] for (p, l) in zip(pred_seq, label_seq) if l != -100]
        for pred_seq, label_seq in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Шаг 5. DataCollator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)



training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,
    num_train_epochs=1,  
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    fp16=True,  # Используем смешанную точность
)

# Шаг 7. Обучение с использованием Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



Train size: 8000, Test size: 2000


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.007000,0.001241,0.999588,0.999588,0.999588,0.999900


TrainOutput(global_step=1000, training_loss=0.1377083191871643, metrics={'train_runtime': 20081.8453, 'train_samples_per_second': 0.398, 'train_steps_per_second': 0.05, 'total_flos': 2090770931712000.0, 'train_loss': 0.1377083191871643, 'epoch': 1.0})

In [5]:
# Шаг 8. Сохранение модели
trainer.save_model("./trained_scibert_ner_model")
tokenizer.save_pretrained("./trained_scibert_ner_model")

print("Model training completed and saved.")


Model training completed and saved.


# Тестирование

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Путь к сохранённой модели
model_path = "./trained_scibert_ner_model"

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)


In [10]:
# Создание NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
# Пример текста для тестирования


# Разметка текста
ner_results = ner_pipeline(example_text)

# Печать результатов
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Entity: had, Label: DATA, Score: 0.5473
Entity: ##oop, Label: DATA, Score: 0.5225
Entity: apache spark, Label: TOOL, Score: 0.9924
Entity: distributed computing, Label: TECHNOLOGY, Score: 0.9979
Entity: principal component analysis, Label: METHOD, Score: 0.9988
Entity: gradient boosting, Label: METHOD, Score: 0.9987
Entity: data, Label: APPLICATION, Score: 0.6238
Entity: processing, Label: METHOD, Score: 0.3217


# NER 

In [ ]:
client = MongoClient('localhost', 27017)
collection = client['VKR1']['proccessed_for_topic_modelling']

In [72]:
# Список доменных терминов (можно расширить)
domain_terms = [
    # Big Data Technologies and Platforms
    'hadoop', 'spark', 'flink', 'storm', 'kafka', 'hive', 'pig', 'cassandra',
    'hbase', 'mongodb', 'redis', 'elasticsearch', 'solr', 'apache beam',
    'apache samza', 'apache apex', 'presto', 'apache drill', 'apache tez',
    'kudu', 'druid', 'nifi', 'zookeeper', 'oozie', 'mahout', 'sqoop', 'ambari',
    'zeppelin', 'mesos', 'alluxio', 'yarn', 'mapreduce', 'azure hdinsight',
    'amazon emr', 'google cloud dataproc', 'cloudera', 'hortonworks', 'databricks',
    'clickhouse', 'apache ignite', 'azure synapse', 'dataproc', 'data fusion',
    'bigquery', 'snowflake', 'redshift', 'minio', 'delta lake', 'lakehouse',
    'trino', 'apache kylin', 'apache carbondata', 'hudi', 'iceberg', 'dremio',
    'bigtable', 'open tsdb', 'prometheus', 'grafana loki', 'vectorized io',
    'openwhisk', 'kubeflow', 'mlflow', 'apache pulsar', 'apache superset',
    'apache ranger', 'datadog', 'new relic', 'dynatrace', 'splunk', 'logstash',
    'fluentd', 'apache arrow', 'delta sharing', 'microsoft fabric', 'aws glue',
    'athena', 'databricks unity catalog', 'citus', 'greenplum', 'hypertable',
    'apache pinot', 'apache heron', 'hazelcast', 'couchbase','ai',

    # Programming Languages and Tools
    'python', 'scala', 'java', 'r', 'sql', 'nosql', 'julia', 'matlab', 'sas',
    'perl', 'go', 'c++', 'rust', 'haskell', 'pig latin', 'hiveql', 'bash',
    'shell scripting', 'php', 'typescript', 'javascript', 'groovy', 'terraform',
    'ansible', 'ruby', 'fortran', 'erlang', 'f#', 'kotlin', 'elixir', 'nim',
    'clojure', 'prolog', 'scheme', 'smalltalk', 'powershell', 'visual basic',
    'tcl', 'awk', 'sed', 'dart', 'cobol', 'vbscript', 'actionscript', 'apl',
    'postgresql', 'mariadb', 'google bigquery sql', 'pl/pgsql', 'solidity',

    # Machine Learning and AI
    'machine learning', 'deep learning', 'clustering', 'classification',
    'regression', 'association rules', 'k-means', 'random forest',
    'gradient boosting', 'svm', 'neural networks', 'decision trees',
    'bayesian methods', 'markov chains', 'genetic algorithms', 'reinforcement learning',
    'exponential smoothing', 'pca', 't-sne', 'lda', 'linear regression',
    'logistic regression', 'naive bayes', 'knn', 'deep neural networks', 'rnn',
    'cnn', 'autoencoders', 'transformers', 'attention mechanism', 'seq2seq models',
    'ensemble learning', 'bagging', 'boosting', 'xgboost', 'lightgbm', 'catboost',
    'hyperparameter tuning', 'grid search', 'bayesian optimization',
    'self-supervised learning', 'contrastive learning', 'meta-learning',
    'few-shot learning', 'multi-task learning', 'generative adversarial networks',
    'variational autoencoders', 'graph neural networks', 'language models',
    'bpe tokenization', 'word2vec', 'fasttext', 'doc2vec', 'sentence transformers',
    'roberta', 'gpt', 'scibert', 'longformer', 'xlm', 'electra', 'albert',
    'vision transformers (vit)', 'bertology', 'zero-shot learning',
    'one-shot learning', 'distillation', 'federated gans', 'tabnet', 'deepar',
    'fastai', 'hugging face transformers', 'pinecone', 'haystack', 'openai apis',
    'langchain', 'llamaindex', 'embeddings',

    # Algorithms and Data Structures
    'bloom filter', 'count-min sketch', 'hyperloglog', 'hashing', 'sharding',
    'partitioning', 'consistent hashing', 'merkle trees', 'trie', 'b-tree',
    'skip lists', 'graph algorithms', 'pagerank', 'giraph', 'pregel', 'gelly',
    'priority queues', 'heap', 'hash tables', 'dynamic programming',
    'shortest path algorithms', 'a-star', 'dijkstra', 'kruskal', 'prim',
    'suffix trees', 'trie structures', 'radix sort', 'red-black trees', 'quadtree',
    'k-d tree', 'segment tree', 'fenwick tree', 'bit masking', 'splay tree',
    'bloomier filters', 'min-heap', 'max-heap', 'hashmaps', 'sparse matrices',
    'dense matrices', 'adjacency lists', 'minimax algorithm',
    'monte carlo tree search', 'simulated annealing', 'tabu search',
    'bellman-ford', 'convex hull', 'suffix arrays', 'lzw compression',
    'sparse neural networks', 'bipartite graphs', 'euler paths',
    'hamiltonian paths', 'approximation algorithms',

    # Application Areas
    'predictive analytics', 'natural language processing', 'computer vision',
    'time series analysis', 'social network analysis', 'bioinformatics',
    'recommender systems', 'internet of things', 'cybersecurity', 'financial analytics',
    'stock market analysis', 'marketing analytics', 'big graph processing',
    'geospatial analysis', 'sentiment analysis', 'digital health', 'personalization',
    'audio analytics', 'robotics', 'autonomous vehicles', 'genomics', 'climate modeling',
    'smart cities', 'energy optimization', 'fraud detection', 'e-commerce analytics',
    'supply chain analytics', 'edge analytics', 'telecommunications analytics',
    'healthcare analytics', 'environmental monitoring', 'disaster prediction',
    'text summarization', 'translation systems', 'video analytics', 'speech recognition',
    'image segmentation', 'retail analytics', 'transportation analytics',
    'insurance analytics', 'gaming analytics', 'agrotech analytics',
    'educational data mining', 'precision medicine', 'quantum computing',
    'remote sensing', 'smart retail', '5g networks analytics', 'streaming media analytics',
    'virtual reality data', 'blockchain data analysis',

    # Concepts and Approaches
    'etl', 'elt', 'stream processing', 'parallel computing', 'distributed systems',
    'in-memory computing', 'lambda architecture', 'kappa architecture', 'cap theorem',
    'base approach', 'acid transactions', 'horizontal scaling', 'vertical scaling',
    'data governance', 'data quality', 'data modeling', 'semantic web', 'microservices',
    'containerization', 'orchestration', 'event-driven architecture',
    'real-time analytics', 'batch processing', 'stream analytics', 'stateful processing',
    'stateless processing', 'low-latency processing', 'adaptive streaming',
    'data wrangling', 'federated learning', 'data pipelines', 'data aggregation',
    'event sourcing', 'data lakes', 'metadata management', 'data versioning',
    'data lineage', 'data observability', 'composable data', 'decentralized data',
    'dataops', 'edge computing', 'digital twins', 'neuro-symbolic ai',
    'knowledge distillation',

    # Data Visualization
    'tableau', 'power bi', 'qlikview', 'd3.js', 'matplotlib', 'seaborn', 'plotly',
    'ggplot2', 'kibana', 'grafana', 'superset', 'dash', 'bokeh', 'gephi',
    'data storytelling', 'heatmaps', 'scatter plots', 'box plots', 'histograms',
    'time series visualization', 'network visualization', 'interactive dashboards',
    'infographics', 'sankey diagrams', 'word clouds', 'sunburst charts', 'radar charts',
    'treemaps', 'correlation plots', 'observable plot', 'rawgraphs', 'flourish',
    'highcharts', 'chart.js', 'piktochart', 'anychart', 'datawrapper', 'infogram',
    'veusz', 'charticulator'

    # Добавленные термины из domain_terms
    'Big Data','large-scale data', 'Data Analysis', 'Data Mining', 'Data Science', 'Artificial Intelligence',
    'MapReduce', 'NoSQL', 'SQL', 'Data Warehousing', 'Business Intelligence',
    'Prescriptive Analytics', 'Descriptive Analytics', 'Apache Hive', 'Apache Pig',
    'Apache Flink', 'Apache Storm', 'Apache Cassandra', 'HBase', 'PyTorch', 'Keras',
    'Scikit-learn', 'Pandas', 'NumPy', 'Data Cleaning', 'Data Integration',
    'AWS', 'Google Cloud Platform', 'Microsoft Azure', 'MLOps', 'Data Pipeline',
    'OLAP', 'OLTP', 'Fog Computing', 'Feature Engineering', 'Dimensionality Reduction',
    'Principal Component Analysis', 'Singular Value Decomposition', 'Data Anonymization',
    'Privacy Preserving Data Mining', 'GDPR', 'Data Ethics', 'Database Management Systems',
    'SQL Server', 'Teradata', 'Data Catalog', 'Data Lineage', 'Data Lakehouse',
    'Structured Data', 'Unstructured Data', 'Semi-Structured Data', 'Distributed Computing',
    'HDFS', 'Columnar Databases', 'Graph Databases', 'Neo4j', 'Data Preprocessing',
    'Data Sampling', 'Data Imputation', 'Anomaly Detection', 'Recurrent Neural Networks',
    'Long Short-Term Memory', 'Attention Mechanisms', 'Data Monetization', 'Data Strategy',
    'Data Literacy', 'Data Democratization', 'Self-Service Analytics', 'Augmented Analytics',
    'Explainable AI', 'AutoML', 'Synthetic Data', 'Data-Driven Decision Making',
    'DataOps', 'Data Stewardship', 'Data Scientist', 'Data Engineer', 'Data Analyst',
    'Data Architect', 'Chief Data Officer', 'Scalability', 'Throughput', 'Fault Tolerance',
    'Load Balancing', 'API', 'RESTful Services', 'Parquet', 'Avro', 'ORC', 'Apache Arrow',
    'Data Storage', 'Data Retrieval', 'ETL Tools', 'Informatica', 'Talend', 'Pentaho',
    'SSIS', 'Data Enrichment', 'Data Warehouse Automation', 'Data Ingestion', 'Data Retention',
    'Data Archiving', 'Data Lifecycle Management', 'Data Replication', 'Master Data Management',
    'Reference Data', 'Data Provenance', 'Data Virtualization', 'Data Federation',
    'Data Consolidation', 'Data Blending', 'Data Cubes', 'Data Mesh', 'Data Fabric',
    'Data Tokenization', 'Data Security', 'Data Privacy', 'Data Loss Prevention',
    'Access Control', 'Authentication', 'Authorization', 'Role-Based Access Control',
    'Identity and Access Management', 'Encryption at Rest', 'Encryption in Transit',
    'SSL', 'TLS', 'Single Sign-On', 'Data Compliance', 'HIPAA', 'PCI DSS', 'Data Breach',
    'Data Incident Response', 'High Availability', 'Serverless Computing',
    'Function as a Service', 'Platform as a Service', 'Infrastructure as a Service',
    'Software as a Service', 'Apache Iceberg', 'Apache Hudi', 'CAP Theorem',
    'Consistency', 'Availability', 'Partition Tolerance', 'ACID', 'BASE', 'CQRS',
    'Message Queues', 'RabbitMQ', 'ActiveMQ', 'ZeroMQ', 'MQTT', 'Streaming Data',
    'Kafka Streams', 'Kinesis', 'Pub/Sub', 'Event Hubs', 'Continuous Integration',
    'Continuous Deployment', 'Agile Methodology', 'Scrum', 'Data Transformation',
    'Data Normalization', 'Standardization', 'One-Hot Encoding', 'Label Encoding',
    'Cross-Validation', 'Random Search', 'K-Fold Cross Validation', 'Ensemble Methods',
    'Stacking', 'Transfer Learning', 'Model Deployment', 'Model Serving', 'Model Monitoring',
    'A/B Testing', 'Concept Drift', 'Model Retraining', 'Model Explainability',
    'SHAP Values', 'LIME', 'Partial Dependence Plots', 'Data Annotation', 'Labeling',
    'Overfitting', 'Underfitting', 'Bias-Variance Tradeoff', 'Regularization',
    'Elastic Net', 'Dropout', 'Early Stopping', 'Mini-Batch Gradient Descent',
    'Adam Optimizer', 'Activation Functions', 'Sigmoid Function', 'Softmax Function',
    'Cost Function', 'Loss Function', 'Backpropagation', 'Epoch', 'Batch Size',
    'Data Silo', 'Edge AI', 'Data Compression', 'UMAP', 'Data Partitioning',
    'Data Sharding', 'Bloom Filters', 'Data Streaming Algorithms', 'Real-Time Analytics',
    'Apache Druid', 'ClickHouse', 'OLAP Cubes', 'Data Marketplace', 'Data Exchange',
    'Data Brokerage', 'Data Licensing', 'Data Contracts', 'Data Quality Metrics',
    'Data Accuracy', 'Data Completeness', 'Data Timeliness', 'Data Conformity',
    'Data Uniqueness', 'Data Validation', 'Data Profiling', 'Data Observability',
    'Data SLOs', 'Anonymization Techniques', 'K-Anonymity', 'Differential Privacy',
    'Homomorphic Encryption', 'Secure Multi-Party Computation', 'Zero-Knowledge Proofs',
    'Blockchain', 'Smart Contracts', 'Consensus Algorithms', 'Proof of Work',
    'Proof of Stake', 'Machine Learning Lifecycle', 'Data Acquisition', 'Data Processing',
    'Model Building', 'Model Evaluation', 'Model Maintenance', 'Experiment Tracking',
    'Weights & Biases', 'Data Privacy Laws', 'CCPA', 'Data Residency', 'Hybrid Cloud',
    'Multicloud', 'Data Compression Techniques', 'Video Analytics', 'Audio Analytics',
    'Topic Modeling', 'Hierarchical Clustering', 'DBSCAN', 'Affinity Propagation',
    'Mean Shift', 'K-Nearest Neighbors', 'Multilayer Perceptron', 'Polynomial Regression',
    'Ridge Regression', 'Lasso Regression', 'Elastic Net Regression', 'ARIMA Models',
    'Autocorrelation', 'Cross-Correlation', 'Spectral Analysis', 'Fourier Transform',
    'Wavelets', 'Mutual Information', 'Chi-Square Test', 'Recursive Feature Elimination',
    'Univariate Selection', 'Canonical Correlation Analysis', 'Discriminant Analysis',
    'Cluster Analysis', 'Cohort Analysis', 'Customer Lifetime Value', 'Churn Prediction',
    'Market Basket Analysis', 'Apriori Algorithm', 'FP-Growth Algorithm', 'Hybrid Recommenders',
    'Matrix Factorization', 'Isolation Forest', 'One-Class SVM', 'Local Outlier Factor',
    'CRISP-DM', 'SEMMA', 'KDD', 'Information Retrieval', 'Named Entity Recognition',
    'Latent Dirichlet Allocation', 'Latent Semantic Analysis', 'Word Embeddings', 'GloVe',
    'BERT', 'Autoencoders', 'Data Masking', 'Data Swapping', 'Re-identification Risk',
    'DAMA-DMBOK', 'DCAM', 'Data Stewardship Council', 'Data Quality Tools',
    'Data Integration Tools', 'Data Governance Tools', 'Data Catalog Tools',
    'Data Lineage Tools', 'Data Masking Tools', 'Data Encryption Tools',
    'Data Backup and Recovery Tools', 'Cloud Storage', 'Object Storage', 'Block Storage',
    'File Storage', 'SAN', 'NAS', 'Storage Tiers', 'Cold Storage', 'Warm Storage',
    'Data Retention Policies', 'Data Disposal', 'Data Destruction', 'Shredding',
    'Degaussing', 'Physical Destruction', 'Data Forensics', 'Incident Response',
    'SIEM', 'SOAR', 'Threat Intelligence', 'Vulnerability Management',
    'Penetration Testing', 'Ethical Hacking', 'Security Awareness Training', 'Phishing',
    'Malware', 'Ransomware', 'Security Monitoring', 'Logging', 'Auditing',
    'Compliance Auditing', 'Regulatory Compliance', 'Data Legislation',
    'Privacy Impact Assessment', 'Risk Management', 'Business Continuity Planning',
    'Green Computing', 'Energy Efficiency', 'Thermal Management', 'Cooling Systems',
    'Virtualization', 'Hypervisors', 'Virtual Machines', 'Containers',
    'Cloud Native Applications', 'Workflow Management', 'Apache Airflow', 'Luigi',
    'Azkaban', 'Data Flow', 'ETL vs ELT', 'Change Data Capture', 'Query Optimization',
    'Cost-Based Optimization', 'Rule-Based Optimization', 'Data Formats', 'YAML',
    'ProtoBuf', 'Thrift', 'Data Serialization', 'Data Deserialization', 'Marshaling',
    'Unmarshaling', 'Operational Data Store', 'Staging Area', 'Sandbox Environment',
    'Data Environments', 'Version Control', 'Branching Strategies', 'Trunk-Based Development',
    'Code Review', 'Pull Requests', 'Merge Requests', 'Build Automation', 'Test Automation',
    'Unit Testing', 'Integration Testing', 'System Testing', 'Acceptance Testing',
    'Regression Testing', 'Test-Driven Development', 'Behavior-Driven Development',
    'API Contracts', 'SLAs', 'SLOs', 'KPIs', 'Metrics', 'Monitoring', 'Alerting',
    'Observability', 'Tracing', 'Metrics Collection', 'Dashboards', 'Reports',
    'Choropleth Maps', 'Bar Charts', 'Line Charts', 'Pie Charts', 'Interactive Visualizations',
    'Natural Language Query', 'Voice Interfaces', 'Chatbots', 'Virtual Assistants',
    'Cognitive Computing', 'Knowledge Graphs', 'Ontologies', 'Taxonomies', 'RDF',
    'SPARQL', 'OWL', 'Open Data', 'Data Sharing', 'FAIR Principles', 'Data Repositories',
    'Data Archives', 'Data Curation', 'Data Lifecycle', 'Data Management Plan',
    'Metadata Standards', 'Dublin Core', 'Data Citation', 'Data Journals', 'Open Science',
    'Data Collaborations', 'Data Consortia', 'Data Partnerships', 'Data Commons',
    'Data Cooperatives', 'Data Crowdsourcing', 'Citizen Science','algorithm'
]

lowercase_terms = [term.lower() for term in domain_terms]

In [94]:
import json
from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Подключение к MongoDB
client = MongoClient('localhost', 27017)
research_paper_collection = client['VKR1']['research_paper']
ner_results_collection = client['VKR1']['ner_results']

# Загрузка данных из MongoDB
def load_data():
    try:
        print("Загрузка данных из MongoDB...")
        research_papers = pd.DataFrame(list(research_paper_collection.find({}, {"_id": 1, "yearPublished": 1})))
        ner_results = pd.DataFrame(list(ner_results_collection.find({}, {"_id": 1, "entities": 1})))
        return research_papers, ner_results
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        return pd.DataFrame(), pd.DataFrame()

research_papers, ner_results = load_data()

# Проверка данных
if research_papers.empty or ner_results.empty:
    print("Нет данных для анализа. Проверьте подключение и содержимое коллекций MongoDB.")
    exit()

# Объединение данных по _id
merged_data = pd.merge(research_papers, ner_results, on="_id")
print(f"Всего объединенных документов: {len(merged_data)}")

# Фильтрация документов с указанным годом публикации
merged_data = merged_data[merged_data["yearPublished"].notnull()]
print(f"Документы с указанным годом публикации: {len(merged_data)}")

# Шаг 1. Группировка текстов по категориям и годам
categories = ['ALGORITHM', 'TECHNOLOGY', 'METHOD', 'DATA', 'MODEL', 'APPLICATION', 'TASK', 'TOOL', 'FRAMEWORK', 'PARAMETER', 'METRIC']
tagged_corpus = {tag: {} for tag in categories}

print("Группировка текстов по категориям и годам...")
for _, row in tqdm(merged_data.iterrows(), total=len(merged_data)):
    year = row["yearPublished"]
    entities = row.get("entities", [])
    if not entities:
        continue
    for entity in entities:
        tag = entity.get("label")
        text = entity.get("text")
        if tag in tagged_corpus and text:
            tagged_corpus[tag].setdefault(year, []).append(text)

# Шаг 2. Определение оптимального числа тем
def find_optimal_topics(texts, vectorizer, min_topics=4, max_topics=15):
    if len(texts) < 5:
        return None, None, None, 0  # Недостаточно данных для анализа
    bow_matrix = vectorizer.fit_transform(texts)
    best_lda = None
    best_topics = None
    best_coherence = -np.inf
    best_num_topics = 0

    for num_topics in range(min_topics, max_topics + 1):
        lda = LatentDirichletAllocation(
            n_components=num_topics,
            max_iter=20,
            random_state=42,
            learning_method="batch"
        )
        lda.fit(bow_matrix)
        coherence = calculate_coherence(lda)
        if coherence > best_coherence:
            best_coherence = coherence
            best_lda = lda
            best_num_topics = num_topics
            feature_names = vectorizer.get_feature_names_out()
            best_topics = {
                f"Topic {i}": [
                    (feature_names[j], topic[j]) for j in topic.argsort()[:-11:-1]
                ]
                for i, topic in enumerate(lda.components_)
            }
    return best_lda, vectorizer, best_topics, best_num_topics

def calculate_coherence(model):
    """
    Вычисляет когерентность как среднюю плотность тем.
    """
    topic_word_distribution = model.components_ / model.components_.sum(axis=1)[:, np.newaxis]
    coherence = np.mean(np.max(topic_word_distribution, axis=1))
    return coherence

# Шаг 3. Тематическое моделирование по годам
def perform_topic_modeling():
    topic_trends = {tag: {} for tag in categories}
    topic_keywords = {tag: {} for tag in categories}
    optimal_topics_count = {tag: {} for tag in categories}

    print("Проведение тематического моделирования...")
    for tag, yearly_texts in tagged_corpus.items():
        print(f"\n--- Анализ категории: {tag} ---")
        vectorizer = CountVectorizer(
            max_features=3000,
            ngram_range=(1, 3),
            stop_words='english',
            token_pattern=r'\b\w+\b',
            min_df=2,
            max_df=0.85
        )

        for year, texts in yearly_texts.items():
            lda, vec, topics, num_topics = find_optimal_topics(texts, vectorizer)
            if lda is not None:
                topic_distribution = lda.transform(vec.transform(texts)).mean(axis=0)
                topic_trends[tag][year] = topic_distribution.tolist()  # Преобразование для сериализации
                topic_keywords[tag][year] = topics
                optimal_topics_count[tag][year] = num_topics

    return topic_trends, topic_keywords, optimal_topics_count

topic_trends, topic_keywords, optimal_topics_count = perform_topic_modeling()

# Шаг 4. Сохранение результатов
def save_results(topic_trends, topic_keywords, optimal_topics_count):
    print("\nСохранение результатов...")
    try:
        with open("topic_trends.json", "w", encoding="utf-8") as f:
            json.dump(topic_trends, f, indent=4)
        with open("topic_keywords.json", "w", encoding="utf-8") as f:
            json.dump(topic_keywords, f, indent=4)
        with open("optimal_topics_count.json", "w", encoding="utf-8") as f:
            json.dump(optimal_topics_count, f, indent=4)
        print("Результаты успешно сохранены.")
    except Exception as e:
        print(f"Ошибка при сохранении результатов: {e}")

save_results(topic_trends, topic_keywords, optimal_topics_count)

# Шаг 5. Визуализация трендов
def plot_trends(topic_trends, optimal_topics_count):
    """
    Визуализация распределения тем по годам.
    """
    for tag, yearly_trends in topic_trends.items():
        if not yearly_trends:
            continue

        # Преобразуем данные в DataFrame
        df_trends = pd.DataFrame.from_dict(yearly_trends, orient="index")
        df_trends.sort_index(inplace=True)

        # Если число топиков варьируется, обрезаем или дополняем недостающие значения
        max_topics = max(optimal_topics_count[tag].values())
        df_trends = df_trends.apply(lambda row: row.tolist() + [0] * (max_topics - len(row)), axis=1)

        plt.figure(figsize=(12, 6))
        for topic_idx in range(max_topics):
            plt.plot(
                df_trends.index,
                df_trends.apply(lambda x: x[topic_idx], axis=1),
                label=f"Topic {topic_idx}"
            )
        plt.title(f"Тренды тем по категории: {tag}")
        plt.xlabel("Год")
        plt.ylabel("Средняя вероятность темы")
        plt.legend()
        plt.grid()
        plt.show()


plot_trends(topic_trends, num_topics=5)


Загрузка данных из MongoDB...
Всего объединенных документов: 86564
Документы с указанным годом публикации: 86564
Группировка текстов по категориям и годам...


100%|██████████| 86564/86564 [00:06<00:00, 13151.31it/s]


Проведение тематического моделирования...

--- Анализ категории: ALGORITHM ---

--- Анализ категории: TECHNOLOGY ---

--- Анализ категории: METHOD ---

--- Анализ категории: DATA ---

--- Анализ категории: MODEL ---

--- Анализ категории: APPLICATION ---

--- Анализ категории: TASK ---

--- Анализ категории: TOOL ---

--- Анализ категории: FRAMEWORK ---

--- Анализ категории: PARAMETER ---

--- Анализ категории: METRIC ---

Сохранение результатов...
Результаты успешно сохранены.


TypeError: plot_trends() got an unexpected keyword argument 'num_topics'